# Galaxy photometry demo

Last verified to run: 2025-02-11

RSP: usdf

Image: w_2024_50

Created by: Melissa Graham

For the photo-z science unit to help get started with a bit of galaxy photometry analysis.

## Import packages

In [ ]:
import lsst.daf.butler as dafButler
import matplotlib.pyplot as plt
import numpy as np

## Butler set up

The recommendation is to use the most recent of the [LSSTComCam Intermittent Cumulative DRP Runs](https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs).

Today, that is the one which uses w_2025_05 and can be found in the butler on `repo/main` in the collection `LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666`.

In [ ]:
collection = "LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666"
butler = dafButler.Butler("/repo/main", collections=collection)
registry = butler.registry
use_skymap = "lsst_cells_v1"

Explore the registry.

In [ ]:
# searchterm = 'objectTable_tract'
# searchterm = 'deepCoadd_*'
# for dt in sorted(registry.queryDatasetTypes(searchterm)):
#     print(dt)

## Get objects for tract 5063 in ECDFS

I got tract 5063 from looking at the Plot Navigator,
at the plots under `objectTableCore_coaddInputCount_SkyPlot`,
and finding the tract that contains most of the ECDFS field at RA~53.

There are definitely ways to search the butler for overlapping tracts, though, just didn't use them here.

In [ ]:
dataId = {'band': 'i', 'tract': 5063, 'skymap': 'lsst_cells_v1'}

In [ ]:
objects = butler.get('objectTable_tract', dataId = dataId)

In [ ]:
# objects?

In [ ]:
# objects

In [ ]:
# objects.columns

Flux columns (but not flags or errors).

In [ ]:
# for col in objects.columns:
#     if (col.find('lux') >= 0) \
#     & (col.find('lag') < 0) \
#     & (col.find('Err') < 0):
#         print(col)

Other columns; in this case, "patch".

In [ ]:
# for col in objects.columns:
#     if (col.find('atch') >= 0):
#         print(col)

Check for the `detect` flags.

In [ ]:
# for col in objects.columns:
#     if (col.find('etect') >= 0):
#         print(col)

Extendedness flags.

In [ ]:
# for col in objects.columns:
#     if (col.find('xtend') >= 0):
#         print(col)

Plot something simple, ra and dec, for this tract.

In [ ]:
fig = plt.figure(figsize=(4, 3))
plt.plot(objects.coord_ra, objects.coord_dec, 'o', ms=1, alpha=0.2, mew=0)
plt.show()

List the patch number for this tract.

In [ ]:
print(np.unique(objects.patch))

## Create magnitude and color arrays

Calibrated fluxes are in nano-Janskies and the conversion is $m = -2.5 \log(f) + 31.4$.

Fluxes < 0.0 do exist and will cause errors.

In [ ]:
def convert_flux_to_mag(fluxes):
    return -2.5 * np.log10(fluxes) + 31.4

def convert_fluxe_to_mage(fluxes, fluxerrs):
    maxfluxes = fluxes + fluxerrs
    mags1 = convert_flux_to_mag(fluxes)
    mags2 = convert_flux_to_mag(maxfluxes)
    return mags1 - mags2

In [ ]:
fluxtypes = ['gaapOptimal','kron','cModel','psf']
filternames = ['u','g','r','i','z','y']

In [ ]:
for fluxtype in fluxtypes:
    for f,filt in enumerate(filternames):
        fluxname = filt + '_' + fluxtype + 'Flux'
        magname = filt + '_' + fluxtype + 'Mag'
        fluxename = filt + '_' + fluxtype + 'FluxErr'
        magename = filt + '_' + fluxtype + 'MagErr'
        objects[magname] = convert_flux_to_mag(objects[fluxname])
        objects[magename] = convert_fluxe_to_mage(objects[fluxname], objects[fluxename])

In [ ]:
for fluxtype in fluxtypes:
    for f,filt in enumerate(filternames):
        if f < 5:
            clrname = filt + filternames[f+1] + '_' + fluxtype + 'Clr'
            clrename = filt + filternames[f+1] + '_' + fluxtype + 'ClrErr'
            magname1 = filt + '_' + fluxtype + 'Mag'
            magname2 = filternames[f+1] + '_' + fluxtype + 'Mag'
            magename1 = filt + '_' + fluxtype + 'MagErr'
            magename2 = filternames[f+1] + '_' + fluxtype + 'MagErr'
            objects[clrname] = objects[magname1] - objects[magname2]
            objects[clrename] = np.sqrt((objects[magename1])**2 +
                                        (objects[magename2])**2)

In [ ]:
# for col in objects.columns:
#     if (col.find('Mag') >= 0):
#         print(col)

## Apply conditions and make some plots

Conditions:
 * `detect_isPrimary` = True (reject duplicate and parent objects)
 * `refExtendedness` = 1

In [ ]:
ix = np.where((objects['detect_isPrimary']) &
              (objects['refExtendedness']) == 1)[0]
print(len(ix), ' out of ', len(objects), ' seem like primary, extended objects')

Again the simple plot of RA vs Dec, but with the conditions applied.

In [ ]:
fig = plt.figure(figsize=(4, 3))
plt.plot(objects['coord_ra'].loc[ix], objects['coord_dec'].loc[ix],
         'o', ms=1, alpha=0.2, mew=0)
plt.show()

How many of the 216613 "galaxies" have i-band fluxes < 0.

In [ ]:
for fluxtype in fluxtypes:
    print(fluxtype, len(np.where(objects['i_'+fluxtype+'Flux'].loc[ix] < 0.0)[0]))

Show, in the $i$-band, the distribution of fluxes (including flux<0).

In [ ]:
fig = plt.figure(figsize=(4, 3))
for fluxtype in fluxtypes:
    plt.hist(objects['i_'+fluxtype+'Flux'].loc[ix],
             bins=100, log=True, histtype='step', label=fluxtype)
plt.legend(loc='upper right')
plt.xlabel('flux')
plt.show()

In [ ]:
print('flux for mag 17, nearing saturation: ', np.power(10, (17.0-31.4)/(-2.5)))

Take out objects measured as "bright" $flux > 575439$ nJy by any measure, or
with $flux < 0$ by any measure.

In [ ]:
maxflux = 575439.0
tx = np.where((objects['i_psfFlux'].loc[ix] < maxflux) &
              (objects['i_kronFlux'].loc[ix] < maxflux) &
              (objects['i_cModelFlux'].loc[ix] < maxflux) &
              (objects['i_gaapOptimalFlux'].loc[ix] < maxflux) & 
              (objects['i_psfFlux'].loc[ix] > 0) &
              (objects['i_kronFlux'].loc[ix] > 0) &
              (objects['i_cModelFlux'].loc[ix] > 0) &
              (objects['i_gaapOptimalFlux'].loc[ix] > 0))[0]
print(len(tx))

In [ ]:
fig = plt.figure(figsize=(4, 3))
for fluxtype in fluxtypes:
    plt.hist(objects['i_'+fluxtype+'Flux'].loc[ix[tx]],
             bins=100, log=True, histtype='step', label=fluxtype)
plt.legend(loc='upper right')
plt.xlabel('flux')
plt.show()

In [ ]:
fig = plt.figure(figsize=(4, 3))
for fluxtype in fluxtypes:
    plt.hist(objects['i_'+fluxtype+'Mag'].loc[ix[tx]],
             bins=100, log=True, histtype='step', label=fluxtype)
plt.legend(loc='upper right')
plt.xlabel('magnitude')
plt.show()

Compare measures, in magnitudes, in $i$-band.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(6, 2))
axs[0].plot(objects['i_psfMag'].loc[ix[tx]],
            objects['i_gaapOptimalMag'].loc[ix[tx]],
            'o', ms=1, alpha=0.2, mew=0)
axs[0].set_xlabel('i_psfMag')
axs[0].set_ylabel('i_gaapOptimalMag')
axs[1].plot(objects['i_psfMag'].loc[ix[tx]],
            objects['i_cModelMag'].loc[ix[tx]],
            'o', ms=1, alpha=0.2, mew=0)
axs[1].set_xlabel('i_psfMag')
axs[1].set_ylabel('i_cModelMag')
axs[2].plot(objects['i_psfMag'].loc[ix[tx]],
            objects['i_kronMag'].loc[ix[tx]],
            'o', ms=1, alpha=0.2, mew=0)
axs[2].set_xlabel('i_psfMag')
axs[2].set_ylabel('i_kronMag')
fig.tight_layout()
fig.show()